In [1]:
import numpy as np
import torch

In [2]:
import gym

In [4]:
B, n_act, n_atoms = 8, 2, 4

In [22]:
action_prob = torch.randn(B, n_act, n_atoms)
action_prob = action_prob.to(dtype=torch.float32)
action_prob.dtype

torch.float32

In [23]:
categorical_Vmin = -1
categorical_Vmax = +1
categorical_n_atoms = n_atoms

atoms = np.linspace(
        categorical_Vmin,
        categorical_Vmax,
        categorical_n_atoms,
    )  # Z

atoms = torch.tensor(atoms)
atoms = atoms.to(dtype=torch.float32)
atoms.dtype

torch.float32

In [24]:
atoms.dtype

torch.float32

In [32]:
print(action_prob.shape)
print(atoms.shape)

torch.Size([8, 2, 4])
torch.Size([4])


In [39]:
torch.sum(action_prob * atoms.reshape(1,1,-1), dim=-1)

tensor([[-0.8630, -0.6759],
        [-0.0263, -0.8739],
        [ 0.3502,  3.4721],
        [ 1.0632,  1.6932],
        [-0.7237,  1.2273],
        [ 1.7661, -1.4455],
        [ 1.4616,  1.4413],
        [ 1.4367, -1.4427]])

In [43]:
torch.allclose(torch.tensordot(action_prob, atoms, dims=1), torch.sum(action_prob * atoms.reshape(1,1,-1), dim=-1))

False

In [40]:
torch.tensordot(action_prob, atoms, dims=1)

tensor([[-0.8630, -0.6759],
        [-0.0263, -0.8739],
        [ 0.3502,  3.4721],
        [ 1.0632,  1.6932],
        [-0.7237,  1.2273],
        [ 1.7661, -1.4455],
        [ 1.4616,  1.4413],
        [ 1.4367, -1.4427]])

In [1]:
torch.tensordot(action_prob, atoms, dims=1).shape

NameError: name 'torch' is not defined

In [5]:
from src.utils.config import Config
from src.agent.DQN import DQNAgent
from src.agent.CategoricalDQN import CategoricalDQNAgent

import gym

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [1]:
game = 'CartPole-v1'

In [3]:
env = gym.make(game, render_mode=None)
C = Config(env)
dqn_agent = CategoricalDQNAgent(config=C)
dqn_agent.env = env

# dqn_agent.transition()
# print("finish training")
# print('=' * 64)
# print("evaluating.....")
# dqn_agent.eval_step(render=True)

In [8]:
config = C

config = config
input_dim = config.input_dim
action_dim = config.action_dim
num_atoms = config.categorical_n_atoms
output_dim = action_dim * num_atoms

atoms = torch.linspace(
    float(config.categorical_Vmin),
    float(config.categorical_Vmax),
    config.categorical_n_atoms,
)  # Z

layer1 = nn.Linear(input_dim, 48)
layer2 = nn.Linear(48, output_dim)
reshape_layer = lambda x: x.view(-1, action_dim, num_atoms)  # Lambda layer for reshaping

In [ ]:
x = torch.randn()

x = F.relu(layer1(x))
x = layer2(x)
x = reshape_layer(x)
x = F.softmax(x, dim=-1)

action_values = torch.tensordot(x, atoms, dims=1)
print("DEBUG: ###: Action values in NN evolution --:", action_values[0])
# Create an index of the max action value in each batch
idx = torch.argmax(action_values, dim=1).to(torch.int64)
idx = idx.view(x.shape[0], 1, 1).expand(-1, -1, num_atoms)

# Adjust the index to: [[0, 1], [1, 0], [2, 1], [3, 1], etc.]
# First number is row (batch) number, second number is the argmax index

#idx = torch.stack((torch.arange(x.shape[0]), idx), dim=1)
#print("DEBUG 5: new idx.shape, idx", idx.shape, idx)
#print("Max index:", idx.max().item())  # Should be 0 or 1 based on x's shape [128, 2, 50]
#print("DEBUG 6: x", x)
#print("DEBUG: 4: action_values.shape, idx.shape, x.shape:", action_values.shape, (idx.unsqueeze(-1)).shape, x.shape)
# Gather probability histogram for actions with max action values
actorNet_output_argmax = torch.gather(x, dim=1, index=idx).squeeze(1)

In [45]:
# Categorical Test

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:


class CategoricalDQNNet(nn.Module):

    def __init__(self, config):
        super(CategoricalDQNNet, self).__init__()
        # TODO subsume the n_observations and n_actions to the config
        self.config = config
        self.input_dim = config.input_dim
        self.action_dim = config.action_dim
        self.num_atoms = config.categorical_n_atoms
        output_dim = self.action_dim * self.num_atoms

        self.atoms = torch.linspace(
            float(config.categorical_Vmin),
            float(config.categorical_Vmax),
            config.categorical_n_atoms,
        )  # Z

        self.layer1 = nn.Linear(self.input_dim, 48)
        self.layer2 = nn.Linear(48, output_dim)
        self.reshape_layer = lambda x: x.view(-1, self.action_dim, self.num_atoms)  # Lambda layer for reshaping

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        x = self.reshape_layer(x)
        x = F.softmax(x, dim=-1)

        action_values = torch.tensordot(x, self.atoms, dims=1)
        print("DEBUG: ###: Action values in NN evolution --:", action_values[0])
        # Create an index of the max action value in each batch
        idx = torch.argmax(action_values, dim=1).to(torch.int64)
        idx = idx.view(x.shape[0], 1, 1).expand(-1, -1, self.num_atoms)

        # Adjust the index to: [[0, 1], [1, 0], [2, 1], [3, 1], etc.]
        # First number is row (batch) number, second number is the argmax index
        
        #idx = torch.stack((torch.arange(x.shape[0]), idx), dim=1)
        #print("DEBUG 5: new idx.shape, idx", idx.shape, idx)
        #print("Max index:", idx.max().item())  # Should be 0 or 1 based on x's shape [128, 2, 50]
        #print("DEBUG 6: x", x)
        #print("DEBUG: 4: action_values.shape, idx.shape, x.shape:", action_values.shape, (idx.unsqueeze(-1)).shape, x.shape)
        # Gather probability histogram for actions with max action values
        actorNet_output_argmax = torch.gather(x, dim=1, index=idx).squeeze(1)
        #print("DEBUG 7: actorNet_output_argmax.shape", actorNet_output_argmax.shape)

        return x, actorNet_output_argmax